Librerías

In [ ]:
from pathlib import Path
import cv2
import json
import numpy as np

In [ ]:
ROOT = Path().resolve().parent
DATA = ROOT / "data"
IM_DIR = DATA / "images"
ROI_DIR = DATA / "roi"

ROI_DIR.mkdir(parents=True, exist_ok=True)

print("Directorio raíz:", ROOT)
print("Imágenes disponibles:")
for p in sorted(IM_DIR.glob("*.jpg")):
    print("  -", p.name)

Editor de ROIs

In [ ]:
_draw_start = None
_rois = []
_img_base = None
_img_display = None

In [ ]:
def _mouse_callback(event, x, y, flags, param):
    global _draw_start, _rois, _img_base, _img_display

    if event == cv2.EVENT_LBUTTONDOWN:

        if _draw_start is None:
            _draw_start = (x, y)
            cv2.circle(_img_display, (x, y), 4, (0, 255, 0), -1)

        else:
            x1, y1 = _draw_start
            x2, y2 = x, y
            _draw_start = None

            x_min, x_max = sorted([x1, x2])
            y_min, y_max = sorted([y1, y2])

            roi_id = f"P{len(_rois)+1:02d}"
            roi = {
                "id": roi_id,
                "points": [
                    [x_min, y_min],
                    [x_max, y_min],
                    [x_max, y_max],
                    [x_min, y_max],
                ],
            }
            _rois.append(roi)

            cv2.rectangle(_img_display, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(_img_display, roi_id, (x_min+3, y_min+15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1,
                        cv2.LINE_AA)


Función principal del editor

In [ ]:
def roi_editor_rect(image_name: str):
    global _draw_start, _rois, _img_base, _img_display

    _draw_start = None
    _rois = []

    img_path = IM_DIR / image_name
    _img_base = cv2.imread(str(img_path))
    if _img_base is None:
        raise FileNotFoundError(f"No se pudo cargar: {img_path}")
    _img_display = _img_base.copy()

    window = f"ROI Editor - {image_name}"
    cv2.namedWindow(window, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(window, _mouse_callback)

    print("\nEditor de ROIs")
    print("  - Primer clic  : esquina superior izquierda")
    print("  - Segundo clic : esquina inferior derecha")
    print("  - s : guardar y salir")
    print("  - r : borrar todo")
    print("  - q / ESC : salir sin guardar\n")

    while True:
        cv2.imshow(window, _img_display)
        key = cv2.waitKey(20) & 0xFF

        if key == ord("s"):
            out = ROI_DIR / f"{Path(image_name).stem}_rois.json"
            with open(out, "w", encoding="utf-8") as f:
                json.dump(_rois, f, indent=2)
            print(f"Guardado: {out} ({len(_rois)} plazas)")
            break

        elif key == ord("r"):
            _rois = []
            _draw_start = None
            _img_display = _img_base.copy()
            print("Reiniciado.")

        elif key == ord("q") or key == 27:
            print("Salida sin guardar.")
            break

    cv2.destroyWindow(window)


Visualización de ROIs

In [ ]:
import matplotlib.pyplot as plt

def show_rois(image_name: str):
    img_path = IM_DIR / image_name
    roi_path = ROI_DIR / f"{Path(image_name).stem}_rois.json"

    img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)

    with open(roi_path, "r", encoding="utf-8") as f:
        rois = json.load(f)

    overlay = img.copy()

    for roi in rois:
        pts = np.array(roi["points"], dtype=np.int32)
        cv2.polylines(overlay, [pts], True, (0, 255, 0), 2)
        cx, cy = pts.mean(axis=0).astype(int)
        cv2.putText(overlay, roi["id"], (cx-10, cy),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    plt.figure(figsize=(12, 6))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title(f"ROIs — {image_name}")
    plt.show()


Uso

In [ ]:
roi_editor_rect("img_01.jpg")

In [ ]:
show_rois("img_01.jpg")